In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MultiLabelBinarizer
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [ ]:
raw_data = pd.read_csv("survey_data.csv", header=1)
df = raw_data.iloc[:, 16:37].copy()
print("Data shape before cleaning:", df.shape)

q_ward = 'Which ward is this survey from?'
q_age = 'What is your age group?'
q_gender = 'What is your gender identity?'
q_race = 'With what ethnicity or race(s) do you most identify? (Check all that apply)'
q_education = 'What is your highest level of education?'
q_income = 'What is your annual household income?'
q_pol_party = 'How would you describe your political affiliation?'
q_conf_dir_us = 'Overall, do you feel that the country is heading in the right direction or the wrong direction?'
q_conf_dir_ri = 'Overall, do you feel that Rhode Island is heading in the right direction or the wrong direction?'
q_conf_pol_care = 'How confident are you that your Rhode Island politicians care about you?'
q_conf_elect_management = 'How confident are you in the ability of local voting officials to manage elections fairly and accurately?'
q_conf_elect_results = 'How confident are you that the results of the national presidential election will accurately reflect the votes cast?'
q_pres_vote = 'Who did you vote for President'
q_excited_vote = 'Were you excited to vote for your presidential candidate?'
q_info_sources = 'Where do you primarily get your information about voting options, such as precinct locations, voting times, early voting, and mail-in ballot options? (Select all that apply)'
q_in_person_reason = 'Why did you choose to vote in-person on election day? (Select all that apply)'

demographics = [q_ward, q_age, q_gender, q_race, q_education, q_income, q_pol_party]
voter_confidence = [q_conf_dir_us, q_conf_dir_ri, q_conf_pol_care, q_conf_elect_management, q_conf_elect_results]
voter_behavior = [q_pres_vote, q_excited_vote, q_info_sources, q_in_person_reason]

required_columns = demographics + voter_confidence + voter_behavior

outcome1 = 'Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?'
outcome2 = 'Would you support Rhode Island implementing ranked choice voting? This would allow voters to rank candidates in order of preference.'

# Replace empty strings with NaN and drop rows missing any required fields
df[required_columns + [outcome1] + [outcome2]] = df[required_columns + [outcome1] + [outcome2]].replace("", np.nan)
df = df.dropna(subset=required_columns + [outcome1] + [outcome2])
print("Data shape after cleaning:", df.shape)

Data shape before cleaning: (558, 21)
Data shape after cleaning: (439, 21)


In [ ]:
age_order = ['18-24', '25-29', '30-39', '40-49', '50-64', '65 or older']
education_order = ['Less than High School', 'High school graduate / GED', 'Some college', '2 year degree', '4 year degree', 'Professional degree', 'Doctorate']
income_order = ['Less than $30,000', '30,000 - 49,999', '50,000 - 100,000', '100,000 - 199,999', '$200,000 or more']
confidence_order = ['Definitely Wrong', 'Somewhat Wrong', 'Unsure', 'Somewhat Right', 'Definitely Right']
excitement_order = ['Not at all Excited', 'Not Very Excited', 'Somewhat Excited', 'Very Excited', 'Extremely Excited']

# a. Demographics
ward = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit_transform(df[[q_ward]])
age = OrdinalEncoder(categories=[age_order],
                     handle_unknown='use_encoded_value',
                     unknown_value=-1).fit_transform(df[[q_age]])
gender = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit_transform(df[[q_gender]])
race_vals = df[q_race].apply(
    lambda x: [item.strip() for item in x.split(',')] if pd.notnull(x) else []
)
race = MultiLabelBinarizer().fit_transform(race_vals)
education = OrdinalEncoder(categories=[education_order],
                           handle_unknown='use_encoded_value',
                           unknown_value=-1).fit_transform(df[[q_education]])
income = OrdinalEncoder(categories=[income_order],
                        handle_unknown='use_encoded_value',
                        unknown_value=-1).fit_transform(df[[q_income]])
pol_party = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit_transform(df[[q_pol_party]])

# b. Confidence variables
conf_dir_us = OrdinalEncoder(categories=[confidence_order],
                             handle_unknown='use_encoded_value',
                             unknown_value=-1).fit_transform(df[[q_conf_dir_us]])
conf_dir_ri = OrdinalEncoder(categories=[confidence_order],
                             handle_unknown='use_encoded_value',
                             unknown_value=-1).fit_transform(df[[q_conf_dir_ri]])
conf_politicians_ri = OrdinalEncoder(categories=[confidence_order],
                                     handle_unknown='use_encoded_value',
                                     unknown_value=-1).fit_transform(df[[q_conf_pol_care]])
conf_local_vot_off = OrdinalEncoder(categories=[confidence_order],
                                    handle_unknown='use_encoded_value',
                                    unknown_value=-1).fit_transform(df[[q_conf_elect_management]])
conf_election_results = OrdinalEncoder(categories=[confidence_order],
                                       handle_unknown='use_encoded_value',
                                       unknown_value=-1).fit_transform(df[[q_conf_elect_results]])

# c. Behavior variables
pres_vote = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit_transform(df[[q_pres_vote]])
excited_vote = OrdinalEncoder(categories=[excitement_order],
                              handle_unknown='use_encoded_value',
                              unknown_value=-1).fit_transform(df[[q_excited_vote]])
info_sources_vals = df[q_info_sources].apply(
    lambda x: [item.strip() for item in x.split(',')] if pd.notnull(x) else []
)
info_sources = MultiLabelBinarizer().fit_transform(info_sources_vals)
inperson_reason_vals = df[q_in_person_reason].apply(
    lambda x: [item.strip() for item in x.split(',')] if pd.notnull(x) else []
)
inperson_reason = MultiLabelBinarizer().fit_transform(inperson_reason_vals)

In [ ]:
# Encode outcome variables
support_map = {
    "Definitely Not": 1,
    "Probably Not": 2,
    "I would need more information to make a decision": 3,
    "Probably": 4,
    "Definitely": 5
}

# y1: Support for Same-Day Voter Registration
y1 = df[outcome1]
y1_ord = y1.map(support_map)

# y2: Support for Ranked Choice Voting
y2 = df[outcome2]
y2_ord = y2.map(support_map)

In [ ]:
X = np.hstack((
    ward,                # 1. Ward (OneHotEncoder)
    age,                 # 2. Age (OrdinalEncoder)
    gender,              # 3. Gender (OneHotEncoder)
    race,                # 4. Race (MultiLabelBinarizer)
    education,           # 5. Education (OrdinalEncoder)
    income,              # 6. Income (OrdinalEncoder)
    pol_party,           # 7. Political Party (OneHotEncoder)
    conf_dir_us,         # 8. Confidence: Direction US (OrdinalEncoder)
    conf_dir_ri,         # 9. Confidence: Direction RI (OrdinalEncoder)
    conf_politicians_ri, # 10. Confidence: Politicians RI (OrdinalEncoder)
    conf_local_vot_off,  # 11. Confidence: Local Voting Officials (OrdinalEncoder)
    conf_election_results, # 12. Confidence: Election Results (OrdinalEncoder)
    pres_vote,           # 13. Pres Vote (OneHotEncoder)
    excited_vote,        # 14. Excited Vote (OrdinalEncoder)
    info_sources,        # 15. Info Sources (MultiLabelBinarizer)
    inperson_reason      # 16. In-person Reason (MultiLabelBinarizer)
))
print("Combined predictor matrix shape:", X.shape)

Combined predictor matrix shape: (439, 50)


In [ ]:
col_labels = []

# 1) Ward
temp_ward_enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
temp_ward_enc.fit(df[[q_ward]])
ward_labels = [f"Ward={cat}" for cat in temp_ward_enc.categories_[0]]
col_labels.extend(ward_labels)

# 2) Age
col_labels.append("Age_OrdinalEncoded")

# 3) Gender
temp_gender_enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
temp_gender_enc.fit(df[[q_gender]])
gender_labels = [f"Gender={cat}" for cat in temp_gender_enc.categories_[0]]
col_labels.extend(gender_labels)

# 4) Race
temp_race_enc = MultiLabelBinarizer()
race_vals_for_label = df[q_race].apply(lambda x: [item.strip() for item in x.split(',')] if pd.notnull(x) else [])
temp_race_enc.fit(race_vals_for_label)
race_labels = [f"Race={cat}" for cat in temp_race_enc.classes_]
col_labels.extend(race_labels)

# 5) Education
col_labels.append("Education_OrdinalEncoded")

# 6) Income
col_labels.append("Income_OrdinalEncoded")

# 7) Political Party
temp_pol_party_enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
temp_pol_party_enc.fit(df[[q_pol_party]])
pol_party_labels = [f"PolParty={cat}" for cat in temp_pol_party_enc.categories_[0]]
col_labels.extend(pol_party_labels)

# 8) Confidence: Direction US
col_labels.append("ConfDirUS_OrdinalEncoded")

# 9) Confidence: Direction RI
col_labels.append("ConfDirRI_OrdinalEncoded")

# 10) Confidence: Politicians RI
col_labels.append("ConfPoliticiansRI_OrdinalEncoded")

# 11) Confidence: Local Voting Officials
col_labels.append("ConfLocalVotingOfficials_OrdinalEncoded")

# 12) Confidence: Election Results
col_labels.append("ConfElectionResults_OrdinalEncoded")

# 13) Pres Vote
temp_pres_vote_enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
temp_pres_vote_enc.fit(df[[q_pres_vote]])
pres_vote_labels = [f"PresVote={cat}" for cat in temp_pres_vote_enc.categories_[0]]
col_labels.extend(pres_vote_labels)

# 14) Excited Vote
col_labels.append("ExcitedVote_OrdinalEncoded")

# 15) Info Sources
temp_info_sources_enc = MultiLabelBinarizer()
info_sources_vals_for_label = df[q_info_sources].apply(lambda x: [item.strip() for item in x.split(',')] if pd.notnull(x) else [])
temp_info_sources_enc.fit(info_sources_vals_for_label)
info_sources_labels = [f"InfoSources={cat}" for cat in temp_info_sources_enc.classes_]
col_labels.extend(info_sources_labels)

# 16) In-person Reason
temp_inperson_reason_enc = MultiLabelBinarizer()
inperson_reason_vals_for_label = df[q_in_person_reason].apply(lambda x: [item.strip() for item in x.split(',')] if pd.notnull(x) else [])
temp_inperson_reason_enc.fit(inperson_reason_vals_for_label)
inperson_reason_labels = [f"InPersonReason={cat}" for cat in temp_inperson_reason_enc.classes_]
col_labels.extend(inperson_reason_labels)

print("\nDetailed column mapping for X:")
for i, label in enumerate(col_labels):
    print(f"X{i+1}: {label}")


Detailed column mapping for X:
X1: Ward=1.0
X2: Ward=3.0
X3: Ward=5.0
X4: Ward=6.0
X5: Ward=7.0
X6: Ward=8.0
X7: Ward=9.0
X8: Ward=11.0
X9: Ward=12.0
X10: Ward=14.0
X11: Ward=15.0
X12: Age_OrdinalEncoded
X13: Gender=Female
X14: Gender=Male
X15: Gender=Non-binary / Other
X16: Gender=Prefer not to say
X17: Race=American Indian or Alaska Native
X18: Race=Asian
X19: Race=Black or African American
X20: Race=Hispanic or Latino/a
X21: Race=Native Hawaiian or Pacific Islander
X22: Race=Other
X23: Race=White
X24: Education_OrdinalEncoded
X25: Income_OrdinalEncoded
X26: PolParty=Democrat
X27: PolParty=Independent
X28: PolParty=Other third party
X29: PolParty=Republican
X30: ConfDirUS_OrdinalEncoded
X31: ConfDirRI_OrdinalEncoded
X32: ConfPoliticiansRI_OrdinalEncoded
X33: ConfLocalVotingOfficials_OrdinalEncoded
X34: ConfElectionResults_OrdinalEncoded
X35: PresVote=Donald Trump
X36: PresVote=Kamala Harris
X37: PresVote=Other
X38: ExcitedVote_OrdinalEncoded
X39: InfoSources=211 Voter Hotline
X40: I

In [ ]:
# ward as input feature
# x1: Ward 1
# x2: Ward 3
# x3: Ward 5
# x4: Ward 6
# x5: Ward 7
# x6: Ward 8
# x7: Ward 9
# x8: Ward 11
# x9: Ward 12
# x10: Ward 14
# x11: Ward 15

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=ward,distr='logit', hasconst=False)
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=ward,distr='logit', hasconst=False)
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.117660
         Iterations: 54
         Function evaluations: 56
         Gradient evaluations: 56
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -490.65
Model:                                                                                                                                                 OrderedModel   AIC:                             1011.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1073.
Date:   

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


In [ ]:
# age as input feature
# x1: Age_OrdinalEncoded
#     (ordinal encoded age; categories: 18-24, 25-29, 30-39, 40-49, 50-64, 65 or older)

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=age,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=age,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.134724
         Iterations: 20
         Function evaluations: 22
         Gradient evaluations: 22
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -498.14
Model:                                                                                                                                                 OrderedModel   AIC:                             1006.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1027.
Date:   

In [ ]:
# gender as input feature
# x1:female
# x2:male
# x3:non-binary
# x4:prefer not to say

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=gender,distr='logit',hasconst=False)
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=gender,distr='logit',hasconst=False)
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.142672
         Iterations: 33
         Function evaluations: 34
         Gradient evaluations: 34
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -501.63
Model:                                                                                                                                                 OrderedModel   AIC:                             1019.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1052.
Date:   

In [ ]:
# race as input feature
# x1: American Indian or Alaska Native
# x2: Asian
# x3: Black or African American
# x4: Hispanic or Latino/a
# x5: Native Hawaiian or Pacific Islander
# x6: Other
# x7: White

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=race,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=race,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.145527
         Iterations: 62
         Function evaluations: 65
         Gradient evaluations: 65
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -502.89
Model:                                                                                                                                                 OrderedModel   AIC:                             1028.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1073.
Date:   

In [ ]:
# education as input feature
# x1: Education_OrdinalEncoded
#     (ordinal encoded education; categories: Less than High School, High school graduate / GED, Some college, 2 year degree, 4 year degree, Professional degree, Doctorate)

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=education,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=education,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.151229
         Iterations: 17
         Function evaluations: 20
         Gradient evaluations: 20
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -505.39
Model:                                                                                                                                                 OrderedModel   AIC:                             1021.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1041.
Date:   

In [ ]:
# income as input feature
# x1: Income_OrdinalEncoded
#     (ordinal encoded income; categories: Less than $30,000, 30,000 - 49,999, 50,000 - 100,000, 100,000 - 199,999, $200,000 or more)

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=income,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=income,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.153110
         Iterations: 18
         Function evaluations: 19
         Gradient evaluations: 19
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -506.22
Model:                                                                                                                                                 OrderedModel   AIC:                             1022.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1043.
Date:   

In [ ]:
# political party as input feature
# x1: Democrat
# x2: Independent
# x3: Other third party
# x4: Republican

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=pol_party,distr='logit',hasconst=False)
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=pol_party,distr='logit',hasconst=False)
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.121658
         Iterations: 32
         Function evaluations: 33
         Gradient evaluations: 33
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -492.41
Model:                                                                                                                                                 OrderedModel   AIC:                             1001.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1033.
Date:   

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


In [ ]:
# confidence: direction US as input feature
# x1: ConfDirUS_OrdinalEncoded
#     (ordinal encoded confidence; categories: Definitely Wrong, Somewhat Wrong, Unsure, Somewhat Right, Definitely Right)

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=conf_dir_us,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=conf_dir_us,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.143501
         Iterations: 18
         Function evaluations: 19
         Gradient evaluations: 19
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -502.00
Model:                                                                                                                                                 OrderedModel   AIC:                             1014.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1034.
Date:   

In [ ]:
# confidence_dir_RI as input feature
# x1: ConfDirRI_OrdinalEncoded
#     (ordinal encoded confidence; categories: Definitely Wrong, Somewhat Wrong, Unsure, Somewhat Right, Definitely Right)

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=conf_dir_ri,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=conf_dir_ri,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.122849
         Iterations: 21
         Function evaluations: 22
         Gradient evaluations: 22
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -492.93
Model:                                                                                                                                                 OrderedModel   AIC:                             995.9
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1016.
Date:   

In [ ]:
# confidence: politicians RI as input feature
# x1: ConfPoliticiansRI_OrdinalEncoded
#     (ordinal encoded confidence; categories: Definitely Wrong, Somewhat Wrong, Unsure, Somewhat Right, Definitely Right)

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=conf_politicians_ri,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=conf_politicians_ri,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.155620
         Iterations: 19
         Function evaluations: 20
         Gradient evaluations: 20
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -507.32
Model:                                                                                                                                                 OrderedModel   AIC:                             1025.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1045.
Date:   

In [ ]:
# confidence: local voting officials as input feature
# x1: ConfLocalVotingOfficials_OrdinalEncoded
#     (ordinal encoded confidence; categories: Definitely Wrong, Somewhat Wrong, Unsure, Somewhat Right, Definitely Right)

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=conf_local_vot_off,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=conf_local_vot_off,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.154944
         Iterations: 20
         Function evaluations: 21
         Gradient evaluations: 21
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -507.02
Model:                                                                                                                                                 OrderedModel   AIC:                             1024.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1044.
Date:   

In [ ]:
# confidence: election results as input feature
# x1: ConfElectionResults_OrdinalEncoded
#     (ordinal encoded confidence; categories: Definitely Wrong, Somewhat Wrong, Unsure, Somewhat Right, Definitely Right)

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=conf_election_results,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=conf_election_results,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.152175
         Iterations: 20
         Function evaluations: 21
         Gradient evaluations: 21
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -505.80
Model:                                                                                                                                                 OrderedModel   AIC:                             1022.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1042.
Date:   

In [ ]:
# pres vote as input feature
# x1: Donald Trump
# x2: Kamala Harris
# x3: Other

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=pres_vote,distr='logit', hasconst=False)
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=pres_vote,distr='logit', hasconst=False)
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.115977
         Iterations: 27
         Function evaluations: 28
         Gradient evaluations: 28
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -489.91
Model:                                                                                                                                                 OrderedModel   AIC:                             993.8
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1022.
Date:   

In [ ]:
# excited to vote as input feature
# x1: ExcitedVote_OrdinalEncoded
#     (ordinal encoded excitement; categories: Not at all Excited, Not Very Excited, Somewhat Excited, Very Excited, Extremely Excited)


# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=excited_vote,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=excited_vote,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.155549
         Iterations: 17
         Function evaluations: 20
         Gradient evaluations: 20
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -507.29
Model:                                                                                                                                                 OrderedModel   AIC:                             1025.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1045.
Date:   

In [ ]:
# voting info sources as input feature
# x1: 211 Voter Hotline
# x2: Friends/Family
# x3: Religious/Community Organizations
# x4: Rhode Island Secretary of State's Website
# x5: Social Media
# x6: The Voter Information Handbook sent to my house

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=info_sources,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=info_sources,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.149136
         Iterations: 36
         Function evaluations: 38
         Gradient evaluations: 38
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -504.47
Model:                                                                                                                                                 OrderedModel   AIC:                             1029.
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1070.
Date:   

In [ ]:
# voting in-person reason as input feature
# x1: I did not feel like I had all the information necessary to vote early or by mail
# x2: I do not trust mail-in voting
# x3: I forgot to vote early or by mail
# x4: I prefer voting on election day
# x5: I wanted to wait to learn more about the candidates
# x6: Today was the best option for my schedule

# same day voter registration model
model_1 = OrderedModel(endog=y1_ord,exog=inperson_reason,distr='logit')
res_1 = model_1.fit(method='bfgs')
print(res_1.summary())

# ranked choice voting model
model_2 = OrderedModel(endog=y2_ord,exog=inperson_reason,distr='logit')
res_2 = model_2.fit(method='bfgs')
print(res_2.summary())

Optimization terminated successfully.
         Current function value: 1.111191
         Iterations: 68
         Function evaluations: 70
         Gradient evaluations: 70
                                                                                            OrderedModel Results                                                                                            
Dep. Variable:     Would you support Rhode Island implementing same day voter registration, allowing unregistered voters to both register and vote on election day?   Log-Likelihood:                -487.81
Model:                                                                                                                                                 OrderedModel   AIC:                             995.6
Method:                                                                                                                                          Maximum Likelihood   BIC:                             1036.
Date:   